### Introduction

In this kernel,I am going to do exploratory data analysis on dataset related to major OTT players.It has information on the list of movies,directors,genere,country etc.A similar analysis was carried out by me on the netflix dataset and I had developed a dashboard.Pls navigate this [link](https://www.kaggle.com/gsdeepakkumar/now-stream-min-ing-on-netflix) to know more.

Let start.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from collections import Counter
warnings.filterwarnings('ignore')
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/movies-on-netflix-prime-video-hulu-and-disney/MoviesOnStreamingPlatforms_updated.csv')

### Basic data quality Checks

In [ ]:
df.head()

In [ ]:
df.shape

We have 16,744 rows and 17 columns.Lets check the columns.

In [ ]:
df.dtypes

Unnamed:0 is a redundant column and provides info on the index of the rows.Lets remove it.Also,there is a column called Type which provides info on whether the title is a movie or a series.In our case,all are movies.Hence we can remove that column aswell.

In [ ]:
df.drop(['Unnamed: 0','Type'],axis=1,inplace=True)

Lets check the status on the null values in each of the columns

In [ ]:
df.isnull().sum()

We have complete information on the titles and in which streaming platform the title is available.Meta informations like director,genres,country,language,IMDB have some missing values.

In [ ]:
len(df['Title'].unique())

There are 16744 titles equal to the number of rows in the dataset.So,there are no duplicate entries.

### Titles Analysis

In [ ]:
print(f'''{df['Netflix'].sum()} titles are streaming in Netflix , {df['Hulu'].sum()} titles are streaming in Hulu, {df['Prime Video'].sum()} titles are streaming on Prime video , {df['Disney+'].sum()} titles are streaming on Disney+''')

We have close to 12k titles which are streaming on Prime video whereas 3.5k titles comes from Netflix.Hulu and Disney+ titles are very less compared to the two.

Are there any titles available in all the 4 streaming platforms ?

In [ ]:
df[(df['Netflix']==1) & (df['Hulu']==1) & (df['Prime Video']==1) & (df['Disney+']==1)]

Well,no movie title is present in all the 4 streaming platforms.Why cant we just take a count of the 4 streaming platforms and check how the titles are distributed across the platforms ?.

In [ ]:
df['OTT_Count']=df['Netflix']+df['Hulu']+df['Prime Video']+df['Disney+']

In [ ]:
(df['OTT_Count'].value_counts()/df.shape[0])*100

So,here are the results.Close to 96 % of the movie titles available in this dataset are present in only one of the 4 streaming platforms.Less than 4 % of the titles are available in 2 platforms whereas it is very rate to see a title available in 3 plaforms.Lets do a quick check on the number of titles and other details for the titles present in 3 streaming platforms.

In [ ]:
df[df['OTT_Count']==3]

* There are 10 movie titles in all present in 3 streaming platforms

* Netflix,Hulu and Prime video seem to have most common titles among the 4 platforms.There is a great overlap between Hulu and Prime video here with each differing only with 1 title.And that two titles - Zapped and Amy seem to be available for streaming in Netflix.

* More Hulu titles are available either in Netflix and Prime video compared to the Disney+.

### Netflix Vs Prime Movies

From the previous analysis,we found that there are 3560 titles available in Netflix & 12354 titles are available in Prime.Let us make a comparison between these two streaming platforms and infer our observations.Pls note that the above count includes those titles which are available to stream in other platforms as well.Our analysis will consider those titles available exclusively in one platform.

In [ ]:
netflix=df[(df['Netflix']==1) & (df['Hulu']==0) &(df['Prime Video']==0)& (df['Disney+']==0)]
netflix.shape

In [ ]:
prime=df[(df['Netflix']==0) & (df['Hulu']==0) &(df['Prime Video']==1)& (df['Disney+']==0)]
prime.shape

In [ ]:
(netflix['Age'].value_counts()/netflix.shape[0])*100

In [ ]:
(prime['Age'].value_counts()/prime.shape[0])*100

* Analysing the age restrictions for the titles,it is seen that both the streaming platforms have 20% of their titles in 18+ rating.
* The percentage of 13+rated movies is higher in netflix compared to Prime.While there is a 4% difference between the 13+ titles there is a 2% difference in 7+ ratings.

Lets check their imdb rating.

In [ ]:
f,ax=plt.subplots(1,2,figsize=(12,5))
sns.distplot(netflix['IMDb'],ax=ax[0])
ax[0].set_title("Distribution of IMDb rating-Netflix Titles",fontsize=15)
ax[0].set_xlabel("IMDb rating",fontsize=8)
ax[0].set_ylabel("Frequency",fontsize=8)
sns.distplot(prime['IMDb'],ax=ax[1])
ax[1].set_title("Distribution of IMDb rating-Prime Titles",fontsize=15)
ax[1].set_xlabel("IMDb rating",fontsize=8)
ax[1].set_ylabel("Frequency",fontsize=8)

In [ ]:
plt.figure(figsize=(8,8))
sns.kdeplot(netflix['IMDb'],shade=True,color='red')
sns.kdeplot(prime['IMDb'],shade=True,color='blue')
plt.legend(title='IMDB Rating',labels=['netflix','prime'])
plt.title("Distribution of Ratings",fontsize=15)
plt.xlabel("IMDB Rating",fontsize=8)


* There is a difference in the IMDB ratings between Netflix and Prime.
* The distribution is close to bell shaped curve (not a perfect) with the peak between 6-7 for Netflix and 5-6 for Prime.
* There are few titles having rating greater than 9.
* Compared to Netflix,there are more titles with lesser IMDB ratings (between 2-5 range) available in Prime whereas in the higher ratings scale,Netflix seem to be dominant.

Lets check the ratings from rotten tomatoes.

In [ ]:
netflix['Rotten Tomatoes']=netflix['Rotten Tomatoes'].str.replace(r'%',r'').astype('float')
prime['Rotten Tomatoes']=prime['Rotten Tomatoes'].str.replace(r'%',r'').astype('float')

In [ ]:
f,ax=plt.subplots(1,2,figsize=(12,5))
sns.distplot(netflix['Rotten Tomatoes'],ax=ax[0])
ax[0].set_title("Distribution of Rotten Tomatoes rating",fontsize=15)
ax[0].set_xlabel("Rotten Tomatoes rating",fontsize=8)
ax[0].set_ylabel("Frequency",fontsize=8)
sns.distplot(prime['Rotten Tomatoes'],ax=ax[1])
ax[1].set_title("Distribution of Rotten Tomatoes rating",fontsize=15)
ax[1].set_xlabel("Rotten Tomatoes rating",fontsize=8)
ax[1].set_ylabel("Frequency",fontsize=8)

Rattings in rotten tomatoes have been on the higher side with titles available in the range between 80-100 for both Netflix and Prime.Though there is a difference between the two streaming platforms when it comes to IMDB ratings,similar ratings are seen for the titles available in Prime and Netflix.

Lets check the genres available in this platform

In [ ]:
def clean_column(df,col):
    """
    Function to extract and count the individual entries in a column separated by comma.
    param:
    df : dataframe
    col:column to be counted
    returns:
    set containing values available in the column
    counter object with count of each values in the column
    """
    df[col]=df[col].astype('str')
    empty_set=set()
    count_values=Counter()

    for gen in df[col]:
        gen=gen.split(',')
        empty_set.update([g.strip() for g in gen])
        for g in gen:
            count_values[g.strip()]+=1
            
    return empty_set,count_values

In [ ]:
genres,count_genres=clean_column(netflix,'Genres')
genres_prime,count_genres_prime=clean_column(prime,'Genres')

In [ ]:
def clean_data(data,dictval):
    """
    A function to create a dataframe from dictionary object for plotting
    params:
    dictval:dictionary
    data:dataframe for analysis
    returns:
    dataframe
    """
    df=pd.DataFrame.from_dict(dictval,orient='index') # keys should be rows.
    df.sort_values(0,ascending=False,inplace=True)
    df.rename(columns={0:'count'},inplace=True)
    df['perc']=(df['count']/data.shape[0])*100
    return df

In [ ]:
netflix_genres=clean_data(netflix,count_genres)
prime_genres=clean_data(prime,count_genres_prime)

In [ ]:
netflix_genres

In [ ]:
prime_genres

In [ ]:
f,ax=plt.subplots(1,2,figsize=(12,8))
sns.barplot(netflix_genres[:10].index,y='perc',data=netflix_genres[:10],ax=ax[0])
ax[0].set_xlabel("Genres",fontsize=8)
ax[0].set_ylabel("Percengage of titles(%)",fontsize=8)
ax[0].set_title("Genres in Netflix",fontsize=15)
ax[0].tick_params(labelrotation=90)
sns.barplot(prime_genres[:10].index,y='perc',data=prime_genres[:10],ax=ax[1])
ax[1].set_xlabel("Genres",fontsize=8)
ax[1].set_ylabel("Percengage of titles(%)",fontsize=8)
ax[1].set_title("Genres in Prime",fontsize=15)
ax[1].tick_params(labelrotation=90)

* The top 3 generes available in Netflix and Prime are the same.
* Though the difference in percentage is relatively large for the first three genres,it is less for the subsequent genres.They seem to be available in equal percentages.
* Though the percentage of drama titles are very close between Netflix and Prime,more titles in Comedy genre are available in Netflix comared to Prime whereas more titles in Thriller genre are available to stream in Prime compared to Netflix.
* While 16% of the movie titles belong to Romance genre in Netflix,an equal percentage of titles belonging to Action genre is available in Prime.
* Mystery genre of movies seem to occupy the 10th place whereas the genre is not in top 10 for Netflix.

So,when it comes to genres,both Netflix and Prime have almost equal representation from Drama,Comedy and Thriller.


# Conclusion

In this notebook , I have analysed the list of movies available in popular OOT platforms like Hotstar,Netflix & Prime.

1.96 % of the titles listed in this dataset are available to view in only one platform.Therefore , each platform has got is own unique catalog of movies and genres to pull its customers.

2.Netflix has got more movies with higherIMDB rating of 6-7 when compared to Prime.This can be noticed from the histogram plot of generes between Netflix and prime.But when it comes to ratings on rotten tomato,both the platforms have got similar ratings for the titles.

3.When it comes to generes,Drama , Comedy and Thriller are the most available generes in both Netflix and Prime.
